In [1]:
# !pip install crewai crewai_tools langchain_community

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew, LLM

In [4]:
choice = 1
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()

if choice==1:
    api_key = os.getenv("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = api_key    
    llm = ChatOpenAI(model = "gpt-4o-mini",openai_api_key=api_key)

elif choice ==2:
    
    os.environ["OPENAI_API_KEY"] = "NA"
    llm = ChatOpenAI(
        model = "ollama/llama3.2",
        base_url = "http://localhost:11434"
    )
# else:
#     from langchain_community.llms import HuggingFaceHub
#     llm = HuggingFaceHub(
#         repo_id="HuggingFaceH4/zephyr-7b-beta",
#         huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
#         task="text-generation",
#     )

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

serper_key = os.getenv("SERPER_key")
os.environ["SERPER_API_KEY"] = serper_key

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [9]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [10]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent ],
    
    tasks=[data_analysis_task, 
           strategy_development_task ],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

In [11]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'NVDA',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
}

In [12]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (NVDA). Use statistical modeling and machine learning to identify trends and predict market movements.
 

I encountered an error while trying to use the tool. This was the error: 2 validation errors for DelegateWorkToolSchema
task
  Input should be a valid string [type=string_type, input_value={'description': 'Continuo...ements.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
context
  Input should be a valid string [type=string_type, input_value={'description': 'The purp... price.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'typ

In [14]:
from IPython.display import display, Markdown
Markdown(result.raw)

**Trading Strategies for NVIDIA (NVDA)**

1. **Momentum Trading Strategy**:
   - **Objective**: Capitalize on NVDA's recent upward momentum.
   - **Entry Point**: Buy NVDA stock when it breaks above $142.51 with increased trading volume.
   - **Exit Point**: Sell when the stock increases by 5-7% or shows signs of weakening momentum.
   - **Risk Management**: Set a stop-loss at 2% below the entry point to limit potential losses.

2. **Earnings Report Positioning**:
   - **Objective**: Prepare for volatility surrounding upcoming earnings reports.
   - **Pre-Earnings Strategy**: Buy before the earnings announcement if analyst ratings are positive, specifically if the stock is trending upward.
   - **Post-Earnings Action**: If earnings exceed expectations, hold for an additional 1-3 days. If they miss expectations, sell rapidly to minimize losses.
   - **Risk Management**: Implement a trailing stop-loss that adjusts upward as the stock price increases to protect profits.

3. **Range Trading Strategy**:
   - **Objective**: Utilize the historical trading range of NVDA.
   - **Buy Zone**: Purchase NVDA stock when it drops to the lower bound of the trading range (approximately $130).
   - **Sell Zone**: Sell when it reaches the upper bound of the trading range (approximately $150).
   - **Risk Management**: Use a stop-loss set at 2% below the lower bound purchase price or a fixed dollar amount based on portfolio size.

4. **5-Day Moving Average Crossover**:
   - **Objective**: Use the moving average as a trend indicator.
   - **Buy Signal**: Enter a buy position when the 5-day moving average crosses above the 20-day moving average.
   - **Sell Signal**: Exit the position when the 5-day moving average crosses below the 20-day moving average.
   - **Risk Management**: Use a stop-loss order at 3% below the purchase price.

5. **Volume-Based Strategy**:
   - **Objective**: Focus on trading volume as an indicator of bullishness.
   - **Entry Point**: Buy when trading volume is 50% greater than the average volume over the last 30 days, indicating strong buying interest.
   - **Exit Point**: Sell if volume drops significantly below the average, signaling a lack of interest and potential downturn.
   - **Risk Management**: Establish a stop-loss at 1.5% below the entry price.

**Conclusion**: Each strategy aligns with a medium risk tolerance while capitalizing on the strengths of NVIDIA’s stock performance and attributes. It's crucial to continuously monitor the indicators mentioned and adapt strategies as necessary based on market conditions and company developments.